# CS3802--Machine Learning Algorithms Lab

Adithya V |
BTech CSE (IoT) - A | 21011102009

## Exercise 7
---
Use the given dataset and implement K-Means from scratch and use the sklearn K-Means implementation. Compare the results of both the implementations and write your inferences in the ipynb file itself.



### Importing the necessary libraries and reading the dataset

In [1]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')
df= pd.read_csv(r'data.csv')
df.head()

c:\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [2]:
df.columns

Index(['CustomerID', 'Gender', 'Age', 'Annual Income (k$)',
       'Spending Score (1-100)'],
      dtype='object')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


### Checking for null values

In [4]:
df.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

In [5]:
df = df.drop(['CustomerID'], axis = 1)
df['Annual Income (k$)'] = 1000 * df['Annual Income (k$)']
df.rename(columns = {'Annual Income (k$)': 'Annual Income', 'Spending Score (1-100)': 'Spending Score'}, inplace = True)
print('There are {} missing values in the dataframe'.format(df.isna().sum().sum()))
df.head()

There are 0 missing values in the dataframe


,Gender,Age,Annual Income,Spending Score
0,Male,19,15000,39
1,Male,21,15000,81
2,Female,20,16000,6
3,Female,23,16000,77
4,Female,31,17000,40


### EDA

In [6]:
summary = pd.DataFrame(df.describe())
summary = summary.style.background_gradient(cmap = 'Blues') \
          .set_table_attributes("style = 'display: inline'") \
          .set_caption('Statistics of the Dataset') \
          .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
          }])
summary

,Age,Annual Income,Spending Score
count,200.000000,200.000000,200.000000
mean,38.850000,60560.000000,50.200000
std,13.969007,26264.721165,25.823522
min,18.000000,15000.000000,1.000000
25%,28.750000,41500.000000,34.750000
50%,36.000000,61500.000000,50.000000
75%,49.000000,78000.000000,73.000000
max,70.000000,137000.000000,99.000000


In [8]:

correlations = df.corr()
x = correlations.columns.tolist()
y = correlations.index.tolist()
z = correlations.values

fig = ff.create_annotated_heatmap(x = x, y = y, z = z, 
                                  annotation_text = z.round(2), 
                                  colorscale = 'teal',
                                  hovertemplate = "Correlation of %{x} and %{y}= %{z:.2f}")

fig.update_layout(
    title = dict(
         text = '<b>Feature Correlation</b>',
         x = 0.5,
         y = 0.95,
         font = dict(
             size = 20
         )
    ),
    template = 'plotly_dark',
    width = 800,
    height = 800
)

fig.show()

### Data Pre-Processing

In [9]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
X = df
print(X)

     Gender  Age  Annual Income  Spending Score
0         1   19          15000              39
1         1   21          15000              81
2         0   20          16000               6
3         0   23          16000              77
4         0   31          17000              40
..      ...  ...            ...             ...
195       0   35         120000              79
196       0   45         126000              28
197       1   32         126000              74
198       1   32         137000              18
199       1   30         137000              83

[200 rows x 4 columns]


### K-Means Elbow Method

In [10]:
df['Annual Income'] = df['Annual Income'] / 1000 # reduce Y-axis from B to K

In [11]:
distortion = {}

for i in range(1, 16):
    km = KMeans(n_clusters = i, init = 'k-means++', random_state = 0).fit(X)
    distortion.update({i: km.inertia_})

number_of_clusters = list(distortion.keys())
inertia = list(distortion.values())
    
fig = px.line(x = number_of_clusters, y = inertia, markers = True)

fig.update_layout(template = 'plotly_dark',
                  width = 800, 
                  height = 800,
                  title = dict(
                  text = '<b>Elbow Curve</b>',
                     x = 0.5,
                     y = 0.95,
                     font = dict(
                         size = 20
                         )
                  ),
                  xaxis_title = '<b>Number of Clusters</b>',
                  yaxis_title = '<b>Inertia</b>'
)

fig.update_traces(marker = dict(
    size = 7, 
    color = '#B7E0F7')
)

fig.add_annotation(x = 5, y = 75000, 
                   showarrow = True, 
                   text = 'Optimal Clusters', 
                   arrowhead = 1, 
                   arrowsize = 2,
                   ax = 7,
                   ay = -100,
                   bordercolor = '#FFFF01',
                   borderpad = 3
                  )

fig.show()

### K-Means using sklearn

In [12]:
from sklearn.cluster import KMeans
import plotly.express as px
import numpy as np

# Assuming you have defined X, your data points, somewhere before this code

distortion = {}

for i in range(1, 16):
    km = KMeans(n_clusters=i, init='k-means++', random_state=0).fit(X)
    distortion.update({i: km.inertia_})

number_of_clusters = list(distortion.keys())
inertia = list(distortion.values())
    
fig = px.line(x=number_of_clusters, y=inertia, markers=True)

fig.update_layout(template='plotly_dark',
                  width=800, 
                  height=800,
                  title=dict(
                      text='<b>Elbow Curve</b>',
                      x=0.5,
                      y=0.95,
                      font=dict(
                          size=20
                      )
                  ),
                  xaxis_title='<b>Number of Clusters</b>',
                  yaxis_title='<b>Inertia</b>'
)

fig.update_traces(marker=dict(
    size=7, 
    color='#B7E0F7')
)

fig.add_annotation(x=5, y=75000, 
                   showarrow=True, 
                   text='Optimal Clusters', 
                   arrowhead=1, 
                   arrowsize=2,
                   ax=7,
                   ay=-100,
                   bordercolor='#FFFF01',
                   borderpad=3
                  )

fig.show()


In [14]:

distortion1 = {}
for i in range(1, 16):
    km = KMeans(n_clusters=i, init='k-means++', random_state=0).fit(X)
    distortion1.update({i: km.inertia_})

number_of_clusters1 = list(distortion1.keys())
inertia1 = list(distortion1.values())

fig1 = px.line(x=number_of_clusters1, y=inertia1, markers=True)
fig1.update_layout(
    template='plotly_dark',
    width=400,
    height=400,
    title=dict(
        text='<b>Elbow Curve (Using Plotly Express)</b>',
        x=0.5,
        y=0.95,
        font=dict(size=16)
    ),
    xaxis_title='<b>Number of Clusters</b>',
    yaxis_title='<b>Inertia</b>'
)

# Using plotly graph objects (second implementation)
distortion2 = {}
for i in range(1, 16):
    km = KMeans(n_clusters=i, init='k-means++', random_state=0).fit(X)
    distortion2.update({i: km.inertia_})

number_of_clusters2 = list(distortion2.keys())
inertia2 = list(distortion2.values())

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=number_of_clusters2, y=inertia2, mode='lines+markers', name='Inertia', marker=dict(color='#B7E0F7')))
fig2.update_layout(
    template='plotly_dark',
    width=400,
    height=400,
    title=dict(
        text='<b>Elbow Curve (Using Plotly Graph Objects)</b>',
        x=0.5,
        y=0.95,
        font=dict(size=16)
    ),
    xaxis_title='<b>Number of Clusters</b>',
    yaxis_title='<b>Inertia</b>'
)

# Combine both plots
fig_combined = go.Figure()
fig_combined.add_traces(fig1.data + fig2.data)
fig_combined.update_layout(
    title='<b>Comparison of Elbow Curve Generation Methods</b>',
    width=900,
    height=400,
    template='plotly_dark',
    xaxis_title='<b>Number of Clusters</b>',
    yaxis_title='<b>Inertia</b>'
)
fig_combined.show()


### Conclusion

* In terms of the Elbow Method's application for determining the optimal number of clusters, both plots suggest that the optimal number of clusters could be around 5, where the inertia begins to decrease at a slower rate.
* Both implementations produce similar Elbow Curves, showing the relationship between the number of clusters and the inertia (within-cluster sum of squares).
* The shapes of the curves generated by both methods are nearly identical, indicating that both methods are effectively calculating the inertia for different cluster counts.